#3D Augmented Reality Final Project (A.Y. 2022/2023)


Project 6: Local Features Compression Using Autoencoders
---
**University of Padua**<br>
*Master Degree in ICT for Internet and Multimedia - Cybersystems*
<br>
<br>
**Students:** 
<br>


*   *Gianpietro Nicoletti 2053042*
*   *Ege Alp Turkyener 2049576*
<br>

The resulted data can be found here(i.e. the datasets + the list of keypoints and the matches with all the possible versions: original, compressed, reconstructed):

[RESULTS](https://drive.google.com/drive/folders/1wd_uG5kRr639V9tIZgj11OiLidcqUAb8?usp=sharing)

A short report about this project can be found here: <br>[REPORT](https://drive.google.com/file/d/1Gi7JMtolp5CzvNOMU6bjaM2G6Xwokog7/view?usp=sharing)

## Import of the libraries

In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
!pip install pycolmap
import pycolmap
from PIL import Image, ImageOps

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Conv2DTranspose, Flatten, Reshape, LeakyReLU, Dropout
from keras.models import Model, Sequential
from tensorflow.keras import losses
from keras.utils import Sequence, plot_model

from tqdm import trange
import random

import cv2
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.1 MB/s eta 0:00:00


In [ ]:
PURPLE = '\033[95m'
CYAN = '\033[96m'
DARKCYAN = '\033[36m'
BLUE = '\033[94m'
GREEN = '\033[92m'
YELLOW = '\033[93m'
RED = '\033[91m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

## Downloading of the datasets

In [ ]:
#downloading the datasets
!pip install -U --no-cache-dir gdown --pre
!pip install --upgrade gdown
import gdown

#train set
url = "https://drive.google.com/u/0/uc?id=1_6Jd7BqD7VLhXHGF4ez4AbGHJm-lmRIN&export=download"
output = "castle-P30.zip"
gdown.download(url, output)

url = "https://drive.google.com/u/0/uc?id=1c1dFm0Ci_zMiIJrb9WfZ4xOfNA3IG0D_&export=download"
output = "castle-P19.zip"
gdown.download(url, output)

#test set
url = "https://drive.google.com/u/0/uc?id=19L9VELLy0s2q8wsevkWq1n0VCfXAyQ1h&export=download"
output = "fountain-P11.zip"
gdown.download(url, output)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/u/0/uc?id=1_6Jd7BqD7VLhXHGF4ez4AbGHJm-lmRIN&export=download
To: /content/castle-P30.zip
100%|██████████| 81.2M/81.2M [00:02<00:00, 31.0MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1c1dFm0Ci_zMiIJrb9WfZ4xOfNA3IG0D_&export=download
To: /content/castle-P19.zip
100%|██████████| 51.4M/51.4M [00:01<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=19L9VELLy0s2q8wsevkWq1n0VCfXAyQ1h&export=download
To: /content/fountain-P11.zip
100%|██████████| 32.9M/32.9M [00:03<00:00, 8.24MB/s]


'fountain-P11.zip'

In [ ]:
#unzipping the folders and remove useless files

#train set
!unzip castle-P30.zip
!rm /content/castle-P30/images/K.txt
!unzip castle-P19.zip
!rm /content/castle-P19/images/K.txt

#test set
!unzip fountain-P11.zip
!rm /content/fountain-P11/images/K.txt

Archive:  castle-P30.zip
   creating: castle-P30/gt_dense_cameras/
  inflating: castle-P30/gt_dense_cameras/0000.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0001.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0002.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0003.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0004.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0005.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0006.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0007.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0008.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0009.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0010.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0011.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0012.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0013.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0014.jpg.camera  
  inflating: castle-P30/gt_dense_cameras/0015.jp

## Definition of the utility functions

In [ ]:
def compute_dataset(folders,filename = None):

  """
  Parameters
  ----------
    folders : list or array like
        List of folders where the images are

    filename : str, optional
        A specific file inside the folder
  
  Returns
  ----------
    dataset: list
        All the keypoint,scores and descriptors for the given images inside the folders (or the specified image)
  
  """
  #ERROR
  if(len(folders) != 1 and filename is not None):

    print("You can select only a specific image in a specific folder!")
    return None


  dataset = []

  #compute for all the images inside the folder
  if(filename is None):

    for folder in folders:
      # Get a list of files in the folder
      files = os.listdir(folder)

      
      # Iterate over the files
      for i in trange(len(files)):
          
          file = files[i]
          # Construct the full path to the file
          file_path = os.path.join(folder, file)

          #checking if the path is a file or a directory
          if os.path.isdir(file_path):
            continue

          img = Image.open(file_path).convert('RGB')
          img = ImageOps.grayscale(img)
          img = np.array(img).astype(np.float) / 255

          sift = pycolmap.Sift()

          keypoints, scores, descriptors = sift.extract(img)

          dataset.append([keypoints, scores, descriptors])
      
  #compute for a specific image
  else:

    for i in trange(len(folders)):
      file_path = os.path.join(folders[0], filename)

      img = Image.open(file_path).convert('RGB')
      img = ImageOps.grayscale(img)
      img = np.array(img).astype(np.float64) / 255

      sift = pycolmap.Sift()

      keypoints, scores, descriptors = sift.extract(img)

      dataset.append([keypoints, scores, descriptors])

  return dataset

In [ ]:
def descriptors_toFile(folder,filename, keypoints, descriptors):


  """
  A funtion to create dummy file into a file  in the format keypoints + descriptors (descriptors are all 0's)

  Parameters
  ----------
    folders : str
        folder where to save the file

    filename : str
        name of the file to save inside the folder

    keypoints : list or array like
        list of keypoints 

    descriptors : list or array like (not used)
        list of descriptors
  """

  with open(folder+filename,"w") as data_file:

    #first row is the number of keypoints and the lenght of the descriptors
    n_keypoints = str(len(keypoints))
    len_features = str(descriptors.shape[1])
    header = n_keypoints+" "+len_features+"\n"
    data_file.write(header)

    #one row for each keypoint
    for i in trange(len(keypoints)):
      
      # position (u,v) + scale + orientation 
      u = str(keypoints[i][0])
      v = str(keypoints[i][1])
      s = str(keypoints[i][2])
      o = str(keypoints[i][3])
      point = u + " " + v + " " + s + " " + o + " "
    
      #in the same row the element of the descriptor
      
      row = point #empty string
      for j in range(127):
        
        row = row + str(0)+" " #element + whitespace

      row = row + str(0)+"\n" #instead for the last element we need to put a break line

      data_file.write(row)

## Processing of the datasets 

In [ ]:
#train set
train_folders = ["/content/castle-P30/images/","/content/castle-P19/images/"]

train = compute_dataset(train_folders)

#test set
test_folders = ["/content/fountain-P11/images/"]

test = compute_dataset(test_folders)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-5-ea1ddb9f8dce>:48: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img = np.array(img).astype(np.float) / 255
100%|██████████| 11/11 [00:34<00:00,  3.18s/it]


Here the order and the division doens't matter since these lists are used only for the training and the test and not for the matching phase.

In [ ]:
train_descriptors = []

for i in trange(len(train)):

  for descriptor in train[i][2]:
    
    train_descriptors.append(descriptor)

test_descriptors = []

for i in trange(len(test)):

  for descriptor in train[i][2]:
    
    test_descriptors.append(descriptor)

100%|██████████| 11/11 [00:00<00:00, 564.22it/s]


## Autoencoder definition

In [ ]:
class AutoEncoder(Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = tf.keras.Sequential([
                  Input(128),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(8, activation="relu")
              ])
    self.decoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(16, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(128, activation="relu")
              ])
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
model = AutoEncoder()
model.compile(optimizer="adam", loss = keras.metrics.mean_squared_error)

### Training and Testing

In [ ]:
x_batches = np.array_split(np.array(train_descriptors), 200) #batch training 

epochs = 20
for epoch in range(epochs):

  # Loop over the batches of data
  losses = []
  for x_batch in x_batches:
    
    loss = model.train_on_batch(x_batch, x_batch)
    losses.append(loss)
    print(f'Epoch {epoch+1}, batch loss: {loss}')
  
  print(f'Epoch {epoch+1}, epoch loss: {np.array(losses).mean()}')

In [ ]:
#encoder network
plot_model(model.encoder, to_file='model_plot_encoder.png', show_shapes=True, show_layer_names=False)

In [ ]:
#decoder network
plot_model(model.decoder, to_file='model_plot_decoder.png', show_shapes=True, show_layer_names=False)

In [ ]:
y_batches = np.array_split(np.array(test_descriptors), 200) #batch testing

# Loop over the batches of data
losses = []
for y_batch in x_batches:
  loss = model.test_on_batch(y_batch, y_batch)
  losses.append(loss)
  print(f"Loss: {loss}")

print(f"Mean loss: {np.array(losses).mean()}")


## Computing the descriptors

Notice that, in this part, we recompute the descriptors and the keypoints again because we need to mantain the separation between the datasets. This format is not suitable for the training of the network so we didn't use it before.

In [ ]:
#compute the keypoints and the descriptors for each images in the train set
#and saving them into a dummy file

train = [] #it will contains a list for each dataset used for the train 
for folder in train_folders:

  tmp =[folder] #first element is the folder

  files = os.listdir(folder)

  #Iterate over the files(image)
  for i in trange(len(files)):
    
    file = files[i]
    
    file_path = os.path.join(folder, file)

    #checking if the path is a file or a directory
    if os.path.isdir(file_path):
      continue

    #extracting keypounts and descriptors for the image
    img = Image.open(file_path).convert('RGB')
    img = ImageOps.grayscale(img)
    img = np.array(img).astype(np.float) / 255
    sift = pycolmap.Sift()
    keypoints, scores, descriptors = sift.extract(img)

    #name of the files, keypoints, original descriptors, compressed version, reconstructed version
    tupla = [file,keypoints,descriptors,model.encoder.predict(descriptors),model.predict(descriptors)]
    tmp.append(tupla)

    directory = "dummy_descriptors"
    path = os.path.join(folder, directory)
    if not os.path.exists(path):
      os.mkdir(path)
    descriptors_toFile(path,"/"+file+".txt", keypoints, descriptors)
  
  train.append(tmp) #folder + a list for each image in the folder

In [ ]:
#same comments as above applied to the test

test = []
for folder in test_folders:

  tmp =[folder]

  files = os.listdir(folder)

  #Iterate over the files
  for file in files:
    print(file)
    
    file_path = os.path.join(folder, file)

    #checking if the path is a file or a directory
    if os.path.isdir(file_path):
      continue

    img = Image.open(file_path).convert('RGB')
    img = ImageOps.grayscale(img)
    img = np.array(img).astype(np.float) / 255
    sift = pycolmap.Sift()

    keypoints, scores, descriptors = sift.extract(img)

    tupla = [file,keypoints,descriptors,model.encoder.predict(descriptors),model.predict(descriptors)]
    tmp.append(tupla)
    
    directory = "dummy_descriptors"
    path = os.path.join(folder, directory)
    if not os.path.exists(path):
      os.mkdir(path)
    descriptors_toFile(path,"/"+file+".txt", keypoints, descriptors)
  
  test.append(tmp)

## Finding the matches

### Training set

In [ ]:
# Find the matches between two sets of features original (train)
matcher = cv2.BFMatcher() 

train_original_time = {} #dictionary used to save the execution time

for i in trange(len(train)):

  folder = train[i][0]
  start = time.time() #starting time for this dataset
  with open(folder+"/matches_original.txt","w") as data_file:

    for j in trange(1,len(train[i])):

      for k in range(j+1,len(train[i])):

        name1 = train[i][j][0]
        descriptors1 =  train[i][j][2]

        name2 = train[i][k][0]
        descriptors2 =  train[i][k][2]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance: #excluding false matches 

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n")
                
    end = time.time() #end time
    train_original_time[folder] = end-start #data are saved in the format (key:value): "dataset_name/images/" : time

In [ ]:
# Find the matches between two sets of features compressed (train)
matcher = cv2.BFMatcher() 

train_compressed_time = {}

for i in trange(len(train)):

  folder = train[i][0]
  start = time.time()
  with open(folder+"/matches_compressed.txt","w") as data_file:

    for j in trange(1,len(train[i])):

      for k in range(j+1,len(train[i])):

        name1 = train[i][j][0]
        descriptors1 =  train[i][j][3]
        name2 = train[i][k][0]
        descriptors2 =  train[i][k][3]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance:

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n")

    end = time.time()
    train_compressed_time[folder] = end-start

In [ ]:
# Find the matches between two sets of features after reconstruction (train)
matcher = cv2.BFMatcher()

train_reconstructed_time = {}

for i in trange(len(train)):

  folder = train[i][0]
  start = time.time()
  with open(folder+"/matches_reconstructed.txt","w") as data_file:

    for j in trange(1,len(train[i])):

      for k in range(j+1,len(train[i])):

        name1 = train[i][j][0]
        descriptors1 =  train[i][j][4]

        name2 = train[i][k][0]
        descriptors2 =  train[i][k][4]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance:

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n")

    end = time.time()
    train_reconstructed_time[folder] = end-start

In [ ]:
!zip -r /content/castle-P19_results.zip /content/castle-P19
!zip -r /content/castle-P30_results.zip /content/castle-P30

### Test set

In [ ]:
# Find the matches between two sets of features original (test)
matcher = cv2.BFMatcher()
 
test_original_time = {}

for i in trange(len(test)):

  folder = test[i][0]
  start = time.time()
  with open(folder+"/matches_original.txt","w") as data_file:

    for j in trange(1,len(test[i])):

      for k in range(j+1,len(test[i])):

        name1 = test[i][j][0]
        descriptors1 =  test[i][j][2]

        name2 = test[i][k][0]
        descriptors2 =  test[i][k][2]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance: #excluding false matches 

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n") 
    end = time.time()
    test_original_time[folder] = end-start   

In [ ]:
# Find the matches between two sets of features compressed (test)
matcher = cv2.BFMatcher() 
test_compressed_time = {}
for i in trange(len(test)):

  folder = test[i][0]
  start = time.time()
  with open(folder+"/matches_compressed.txt","w") as data_file:

    for j in trange(1,len(test[i])):

      for k in range(j+1,len(test[i])):

        name1 = test[i][j][0]
        descriptors1 =  test[i][j][3]
        name2 = test[i][k][0]
        descriptors2 =  test[i][k][3]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance:

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n")

    end = time.time()
    test_compressed_time[folder] = end-start 

In [ ]:
# Find the matches between two sets of features after reconstruction (train)
matcher = cv2.BFMatcher() 
test_reconstructed_time = {}
for i in trange(len(test)):

  folder = test[i][0]
  start = time.time()
  with open(folder+"/matches_reconstructed.txt","w") as data_file:

    for j in trange(1,len(test[i])):

      for k in range(j+1,len(test[i])):

        name1 = test[i][j][0]
        descriptors1 =  test[i][j][4]

        name2 = test[i][k][0]
        descriptors2 =  test[i][k][4]

        data_file.write(" \n" + name1 + " " +name2 + "\n")
        
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        for m, n in matches:
            if m.distance < 0.75*n.distance:

                data_file.write(str(m.queryIdx)+" "+str(m.trainIdx)+"\n")

    end = time.time()
    test_reconstructed_time[folder] = end-start

In [ ]:
!zip -r /content/fountain-P11_results.zip /content/fountain-P11

### Checking the execution time during the matching phase

In [ ]:
print(BOLD+"TRAIN SET\n\n"+END)
print("ORIGINAL DESCRIPTORS:\n")
print(train_original_time)
print("\n\nCOMPRESSED DESCRIPTORS:\n")
print(train_compressed_time)
print("\n\nRECONTRUCTED DESCRIPTORS:\n")
print(train_reconstructed_time)

print(BOLD+"\n\n\nTEST SET\n\n"+END)
print("ORIGINAL DESCRIPTORS:\n")
print(test_original_time)
print("\n\nCOMPRESSED DESCRIPTORS:\n")
print(test_compressed_time)
print("\n\nRECONTRUCTED DESCRIPTORS:\n")
print(test_reconstructed_time)